## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-12-22-50-22 +0000,nypost,Teen charged in fatal NYC stabbing of 17-year-...,https://nypost.com/2026/01/12/us-news/teen-cha...
1,2026-01-12-22-47-00 +0000,wsj,"Opinion | ‘I Am Very Expensive,’ Frank Gehry T...",https://www.wsj.com/opinion/i-am-very-expensiv...
2,2026-01-12-22-47-00 +0000,wsj,"Opinion | Lawfare for Dummies, Monetary Edition",https://www.wsj.com/opinion/jerome-powell-subp...
3,2026-01-12-22-46-22 +0000,nyt,Leaders of Historians’ Group Veto Resolutions ...,https://www.nytimes.com/2026/01/12/arts/americ...
4,2026-01-12-22-45-57 +0000,wapo,Arson suspect attacked Mississippi synagogue f...,https://www.washingtonpost.com/nation/2026/01/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,56
47,iran,34
216,fed,22
217,powell,21
251,chair,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
16,2026-01-12-22-35-00 +0000,wsj,The U.S. will put a 25% tariff on any country ...,https://www.wsj.com/politics/policy/u-s-to-imp...,165
176,2026-01-12-16-36-01 +0000,nypost,GOP Sen. Thom Tillis says he will block any of...,https://nypost.com/2026/01/12/business/gop-sen...,152
248,2026-01-12-10-30-00 +0000,wsj,"For years, Fed Chair Jerome Powell responded t...",https://www.wsj.com/economy/central-banking/fo...,151
230,2026-01-12-11-32-54 +0000,nypost,Trump addresses bombshell Jerome Powell invest...,https://nypost.com/2026/01/12/us-news/trump-sa...,144
274,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...,135


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
16,165,2026-01-12-22-35-00 +0000,wsj,The U.S. will put a 25% tariff on any country ...,https://www.wsj.com/politics/policy/u-s-to-imp...
274,135,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...
149,70,2026-01-12-17-59-00 +0000,wsj,Mary Peltola will challenge GOP incumbent Sen....,https://www.wsj.com/politics/elections/mary-pe...
207,66,2026-01-12-13-24-31 +0000,cbc,Swiss court keeps bar owner in custody after d...,https://www.cbc.ca/news/world/swiss-court-bar-...
93,55,2026-01-12-20-01-04 +0000,nypost,Mark Kelly sues Pete Hegseth to stop ‘unconsti...,https://nypost.com/2026/01/12/us-news/mark-kel...
72,47,2026-01-12-20-51-33 +0000,nypost,Minnesota ‘ICE Watch’ group’s manual showed ho...,https://nypost.com/2026/01/12/us-news/minnesot...
229,44,2026-01-12-11-50-09 +0000,nypost,Gut-wrenching footage shows grieving Iranian f...,https://nypost.com/2026/01/12/world-news/foota...
195,41,2026-01-12-14-33-37 +0000,cbc,"Malaysia, Indonesia become 1st countries to bl...",https://www.cbc.ca/news/business/malaysia-indo...
0,37,2026-01-12-22-50-22 +0000,nypost,Teen charged in fatal NYC stabbing of 17-year-...,https://nypost.com/2026/01/12/us-news/teen-cha...
147,37,2026-01-12-18-03-44 +0000,nypost,Meta warns Australia’s under-16 social media b...,https://nypost.com/2026/01/12/business/meta-wa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
